In [50]:
import pandas as pd
import re
import random

i need to create a dataset to train my logistical regression on

i want the following features:

- how many times appears the aliases

- proportion of alphanumeric char

- size of the text

In [3]:
df = pd.read_parquet("data02.parquet")
df.map(lambda x: len(x))

,aliases,api.2000,api.2003,api.2006,api.2009,api.2012,api.2015,api.2018
companies,,,,,,,,
Banco Comercial Português,2,90,191,169,497,983,1219,824
Galp Energia,2,66,233,181,469,964,1104,812
EDP,3,80,245,140,538,1076,1528,872
Sonae,2,130,318,239,459,1109,1400,1026
Mota-Engil,2,3,67,26,164,384,596,445


let's use bcp, 2003 to start

In [23]:
aliases = df.iloc[0,0]
texts = [i["ExtractedText"] for i in df.iloc[0,1]]

my features

In [51]:
def count_alises(aliases, text):
    alias_count = {expression: 0 for expression in aliases}
    for alias in aliases:
        # Use re.escape to handle any special characters in the expression
        pattern = re.escape(alias.lower())
        matches = re.findall(pattern, text.lower())
        alias_count[alias] = len(matches)
    return sum(alias_count.values())

def proportion_alphanumeric(text):
    alphanumeric_chars = sum(char.isalnum() for char in text)
    proportion = alphanumeric_chars / len(text)
    return proportion

def count_dates(text):
    date_pattern = r'\b(\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|\d{4}[-/]\d{1,2}[-/]\d{1,2})\b'
    # 10/11/2024', '10/10/2024', '12-25-1990', '2024-11-05', '01/10/2024'
    dates = re.findall(date_pattern, text)
    date_count = len(dates)
    return date_count

def count_CAPS(text):
    words = text.split()
    uppercase_word_count = sum(1 for word in words if word.isupper())
    return uppercase_word_count

def text_size(text):
    return len(text)

lets start creating the dataset

In [53]:
def dataset_create(aliases, texts, out_name):
    dic = {"news": [], "countALI": [], "propAN": [], "countDTS": [], "txtSZ": [], "countCAPS": [], "txt": []}
    i = 0 
    #global a
    random.shuffle(texts)
    for text in texts:
        valid = int(input(text))
        #valid = int(a[i])
        print(valid, end = ", ")
        if valid not in [0,1,-1,2]:
            print("not 0 or 1")
        elif valid == -1:
            i += 1
            break
        elif valid == 2:
            i += 1
            continue
        elif valid in [0,1]:
            dic["news"].append(valid)
            dic["txt"].append(text)
            dic["countALI"].append(count_alises(aliases, text))
            dic["propAN"].append(proportion_alphanumeric(text))
            dic["countDTS"].append(count_dates(text))
            dic["txtSZ"].append(text_size(text))
            dic["countCAPS"].append(count_CAPS(text))
            i += 1
    pd.DataFrame(dic).to_csv(out_name)
    print(f"\n{i}")


aliases = df.iloc[1,0] #[0,0][1,0]
texts = [i["ExtractedText"] for i in df.iloc[1,2]] #[0,2][1,2]

dataset_create(aliases, texts, "logreg_galp200!.csv")

0, -1, 
2
